<a href="https://colab.research.google.com/github/thronOne97/LEARNING/blob/master/AnalisisSentiemientoOpiniones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Minería de opiniones
## Clasificación de sentimientos en opiniones


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, SimpleRNN, Bidirectional, Dropout
from tensorflow.keras import backend, optimizers, callbacks
from tensorflow.keras.datasets import imdb
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Obtención de los datos.
# Con numeroPalabras establecemos un límite del numero de palabras que tomaremos
# en cuenta. En IMDB las palabras del vocabulario se encuentran ordenadas por frecuencia 
# de aparición. Al asignar num_words en 10,000 indicaremos que 
# consideraremos únicamente las primeras 10,000 palabras que más se repiten.
numeroPalabras = 10000
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words = numeroPalabras)

In [ ]:
# Tamaño del conjunto de entrenamiento.
x_train.shape

In [ ]:
# Así se encuentra codificada una opinión en IMDB


In [ ]:
# La longitud de las opiniones cambia.
len(x_train[13456])

In [ ]:
# Visualización de las etiquetas


## Procesamiento de los datos - Vectorización

In [ ]:
# Diccionario de palabras.
# Con este diccionario accedemos a los índices de las palabra. 
diccionarioPalabras = imdb.get_word_index()
print(len(diccionarioPalabras))

In [ ]:
# Obtención del índice asignado a la palabra "movie".
diccionarioPalabras[]

In [ ]:
# Diccionario para acceder a la palabra a través del índice.
indice2palabra = dict([(indice, palabra) for (palabra, indice) in diccionarioPalabras.items()])

In [ ]:
# La palabra "movie" se encuentra en el lugar 17 de las que más se repiten.


In [ ]:
# ¿Cúal es la palabra que más se usa en las reseñas?
indice2palabra[]

In [ ]:
# Recuerda que a los índices reales de las palabras se les ha sumado 3 
# para evitar confundirlos con los índices 0, 1 y 2 que estan destinados a:
# 0 -> palabras desconocidas
# 1 -> palabra de inicio de todos los vectores de opiniones
# 2 -> palabras fuera de las 10,000 que hemos indicado al inicio del ejercicio
opinionDecodificada = ' '.join(
[indice2palabra.get(i-3, '?') for i in x_train[923]])

In [ ]:
opinionDecodificada

In [ ]:
# Normalización de los vectores de opiniones
def codifica1hot(opiniones, longitud = 10000):
  # La dimensión de opinion1hot matriz será de 25000 filas, 10000 columnas
  opinion1hot = np.zeros((opiniones.shape[0], longitud))
  for opinion in range(opiniones.shape[0]):
    for indice in opiniones[opinion]:
      # Con el 1 indicamos que la palabra asociada a "indice" aparece en "opinion"
      if indice < longitud:
        opinion1hot[opinion, indice] = 1
  return opinion1hot

In [ ]:
# Normalización de los conjuntos de entrenamiento y test
x_train_1hot = codifica1hot(x_train)
x_test_1hot = codifica1hot(x_test)

In [ ]:
# Tamaño del conjunto nuevo de entrenamiento
x_train_1hot

In [ ]:
# Tamaño del conjunto nuevo de test
x_test_1hot

## Generación del modelo

In [ ]:
# Con esta instrucción eliminamos información previa de los modelos.
backend.clear_session()

In [ ]:
# Definición del modelo
modelo = Sequential()


In [ ]:
# Definición del optimizador
Adam = optimizers.Adam(learning_rate=0.001)
# Compilación del modelo
modelo.compile(optimizer=Adam, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Entrenamiento
MLP = modelo.fit(x_train_1hot, y_train, 
               validation_data=(x_test_1hot, y_test), 
               epochs=5, batch_size = 50, 
               shuffle = True, verbose = True)

In [ ]:
# Evaluar las graficas del accuracy (precisión) y del error es una buena práctica.
# Estas graficas te dicen si el modelo necesita más épocas de entrenamiento, 
# si tienes overfitting o si es el óptimo.
plt.plot(MLP.history['accuracy'], label='entrenamiento')
plt.plot(MLP.history['val_accuracy'], label='validación')
plt.legend(loc='best')
plt.xlabel('epoca')
plt.ylabel('accuracy')
plt.ylim([0,1])
plt.show()

## Prueba del modelo

In [ ]:
# Escribe tu opinión
opinion = 

# Vectoriza tu opinión
opinionVector = [int(diccionarioPalabras.get(i,-1) + 3) for i in opinion.split()]

# Estandariza/Codifica tu opinión
opinionArreglo = np.array(opinionVector).reshape(1, len(opinionVector))
opinion1hot = codifica1hot(opinionArreglo)

# Clasifica tu opinión
opinionClasificacion = modelo.predict(opinion1hot)

print(opinionClasificacion)
if opinionClasificacion > 0.5:
  print('opinión positiva')
else:
  print('opinión negativa')


## Preprocesamiento para word embedding

In [ ]:
from keras.preprocessing.sequence import pad_sequences

In [ ]:
max_length = 300 # máximo tamaño de secuencia
padded_x_train = pad_sequences(x_train, maxlen = max_length)
padded_x_test = pad_sequences(x_test, maxlen = max_length)

In [ ]:
# Longitud de los nuevos datos de entrenamiento
padded_x_train

In [ ]:
embedding_dim = 128 # dimensión del vector embedding
max_words = 10000 # número de palabras a considerar

In [ ]:
# Definición del modelo
modeloBi = Sequential()
modeloBi.add(Embedding(input_dim = max_words, output_dim = embedding_dim, input_length = max_length))
modeloBi.add(Bidirectional(LSTM(45)))
modeloBi.add(Dense(64, activation = 'relu'))
modeloBi.add(Dense(1))
modeloBi.summary()

In [ ]:
# Compilación del modelo
modeloBi.compile(optimizer=Adam, loss='binary_crossentropy', metrics=['accuracy'])
# Entrenamiento
RNN = modeloBi.fit(padded_x_train, y_train, validation_data=(padded_x_test, y_test), epochs=5,
               batch_size = 50, shuffle = True, verbose = True)

In [ ]:
# Evaluar las graficas del accuracy (precisión) y del error es una buena práctica.
# Estas graficas te dicen si el modelo necesita más épocas de entrenamiento, 
# si tienes overfitting o si es el óptimo.
plt.plot(RNN.history['accuracy'], label='entrenamiento')
plt.plot(RNN.history['val_accuracy'], label='validación')
plt.legend(loc='best')
plt.xlabel('epoca')
plt.ylabel('accuracy')
plt.ylim([0,1])
plt.show()

In [ ]:
# Escribe tu opinión
opinion = 

# Vectoriza tu opinión
opinionVector = [int(diccionarioPalabras.get(i,-1) + 3) for i in opinion.split()]

# Estandariza/Codifica tu opinión
opinionArreglo = np.array(opinionVector).reshape(1, len(opinionVector))

# Ajusta el vector con padding
opinionPadded = pad_sequences(opinionArreglo, maxlen = max_length)

# Clasifica tu opinión
opinionClasificacion = modeloBi.predict(opinionPadded)
print(opinionClasificacion)
if opinionClasificacion > 0.5:
  print('opinión positiva')
else:
  print('opinión negativa')
